# Case 1 (DSWE)

In [ ]:
import numpy as np
import FINE as fn
import pandas as pd
import os
import re as re
import glob
import xarray
import geopandas as gdp
import matplotlib.pyplot as plt

cwd = os.getcwd()

from getData import getData

data = getData()

%matplotlib inline  
%load_ext autoreload
%autoreload 2


In [ ]:
# Input files
path=r'C:\Users\user\Desktop\Python\Fine-master\My Model\Final Code\InputData'

In [ ]:
names_of_scenarios=['Combined']
print(names_of_scenarios)

for scenario in names_of_scenarios:
    print('##', scenario)
  #  fnameFile.write('{}\n'.format(scenario))
    
    if scenario == 'DSWE':
        DSWE = True
        AWE = False
        desalination = False
        electricity_transmission = True
        electricity_demand = True
        brine_disposal = True
        H2_pipline = True 
        H2_demand = True
        desalinated_water_pipeline = False     
                    
    elif scenario == 'AWE':
        DSWE = False
        AWE = True
        desalination = True
        electricity_transmission = True
        electricity_demand = True
        brine_disposal = True
        H2_pipline = True 
        H2_demand = True
        desalinated_water_pipeline = True
        
    elif scenario == 'Combined':
        DSWE = True
        AWE = True
        desalination = True
        electricity_transmission = True
        electricity_demand = True
        brine_disposal = True
        H2_pipline = True 
        H2_demand = True
        desalinated_water_pipeline = True
                            
                                        
    print('Modelling', scenario, "scenario with:")
    print('DSWE =', DSWE)
    print('AWE =', AWE)
    print('desalination =', desalination)
    print('electricity transmission =', electricity_transmission)
    print('electricity demand =', electricity_demand)
    print('H2 pipline =', H2_pipline)
    print('H2 demand =', H2_demand)
    print('desalinated water pipeline =', desalinated_water_pipeline)

In [ ]:
locations = ['location1', 'location3', 'location4', 'location3_4', 'location5', 'location3_5', 'location6', 'location7', 
             'location8', 'location9', 'location10', 'location11', 'location12', 'location13', 'location15', 'location16',
             'location1_16', 'location17', 'location18', 'location19', 'location20', 'location1_20', 'location21',
             'location22', 'location23', 'location25', 'location26', 'location27', 'location2_70', 'location2_71', 
             'location2_81', 'location3_100', 'location3_101', 'location3_104', 'location3_106', 'location3_107', 
             'location3_103', 'location3_102']

commodities = {'electricity', 'hydrogen', 'seawater', 'brine', 'freshwater'}

commodityUnitsDict={'electricity': 'GW_el', 'hydrogen': 'GW_GH2', 'seawater' : 'Mm3', 
                    'brine' : 'Mm3', 'freshwater' : 'Mm3'}

esM = fn.EnergySystemModel(locations=set(locations), 
                           commodities=commodities,
                           numberOfTimeSteps=8760,
                           commodityUnitsDict=commodityUnitsDict,
                           hoursPerTimeStep=1, 
                           costUnit='1e9 Euro', 
                           lengthUnit='km', 
                           verboseLogLevel=0)

In [ ]:
# Regions on the coasts
eligibility = pd.Series([1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 
                             0, 0, 0, 0, 0, 0, 0, 0], index=locations)

In [ ]:
eligibility

## Source

### PV

In [ ]:
esM.add(fn.Source(esM=esM, 
                  name='PV', 
                  commodity='electricity',
                  hasCapacityVariable=True,
                  operationRateMax=data['PV, operationRateMax'],
                  capacityMax=data['PV, capacityMax'],
                  investPerCapacity=0.52,            # 520€/kw = 0.5billion€/GW
                  opexPerCapacity=0.52 *.017,        # 1.7% of invest
                  interestRate=0.08,
                  economicLifetime=25))

### Full Load Hours

In [ ]:
data['PV, operationRateMax'].sum()

### Seawater

In [ ]:
esM.add(fn.Source(esM=esM, 
                  name='Seawater', 
                  commodity='seawater',
                  hasCapacityVariable=False,
                  locationalEligibility=eligibility
                 ))

## Conversion

### DSW Electrolyzer

In [ ]:
AWE_investPerCapacity = 1
desalination_investPerCapacity = 6.405    # BEuro/GW
desalinatedWater = 327868.85              # amount of desalinated water in m3 for 1 GW_el
lifetime_desalination = 30      
process = 0.27                           # 27% of desalination cost for intake, pretreatment and brine disposal
freshWaterRecovery = 0.45                # 45%

x = [6.405, 8.380, 11.574]
y = [327868.8525, 316455.6962, 253164.557]

for i in [1.139, 1.139*1.2, 1.139*1.4]:
    DSWE_investPerCapacity = i
    print(i)
    
    for j, k in zip(x, y):
        desalination_investPerCapacity = j
        desalinatedWater = k
            
        if DSWE == True:
            capacityMax_DSWE = pd.read_excel(path + '\maxCapacity_DSWE.xlsx', index_col=0, squeeze=True)
            esM.add(fn.Conversion(esM=esM, 
                          name='DSW_Electrolyzer', 
                          physicalUnit='GW_el',
                          commodityConversionFactors={'electricity':-1, 'hydrogen':0.64, # LHV
                                                      'seawater':-172.82/(freshWaterRecovery*1000000), 
                                                      'brine':172.82/((1-freshWaterRecovery*1000000))}, 
                          hasCapacityVariable=True, 
                          locationalEligibility=eligibility,
                          investPerCapacity=(DSWE_investPerCapacity+(process*desalination_investPerCapacity)), 
                          opexPerCapacity=(DSWE_investPerCapacity+(process*desalination_investPerCapacity))*0.025, 
                          #capacityMax=capacityMax_DSWE,   # 5 MW based on production rate = 760 m3_H2/h and 6.6 kWh_el/m3
                          interestRate=0.08,
                          economicLifetime=10))
            
        if AWE == True:
            esM.add(fn.Conversion(esM=esM, 
                          name='AW_Electrolyzer', 
                          physicalUnit='GW_el',
                          commodityConversionFactors={'electricity':-1, 'hydrogen':0.65, # LHV
                                                      'freshwater':-175.52/1000000}, 
                          hasCapacityVariable=True,
                          investPerCapacity=AWE_investPerCapacity,
                          opexPerCapacity=AWE_investPerCapacity*0.025,
                          #capacityMax=0.005,   # 5 MW based on production rate = 760 m3_H2/h and 6.6 kWh_el/m3
                          interestRate=0.08,
                          economicLifetime=10))
                
        if desalination == True:
            capacityMax_Desalination = pd.read_excel(path + '\maxCapacity_Desalination.xlsx', index_col=0, squeeze=True)
            esM.add(fn.Conversion(esM=esM, 
                          name='Desalination plant', 
                          physicalUnit='GW_el',
                          commodityConversionFactors={'electricity':-1, 'freshwater':desalinatedWater/1000000,
                                                      'seawater':-desalinatedWater/(freshWaterRecovery*1000000), 
                                                      'brine':(desalinatedWater*(1-freshWaterRecovery))/(freshWaterRecovery*1000000)}, 
                          hasCapacityVariable=True, 
                          locationalEligibility=eligibility,
                          investPerCapacity=desalination_investPerCapacity,        
                          opexPerCapacity=desalination_investPerCapacity*0.04,  
                          #capacityMax=capacityMax_Desalination,   # 6.7 MW based on production rate = 40000 m3/day and 4 kWh_el/m3
                          interestRate=0.08, 
                          economicLifetime=lifetime_desalination))
            
            # Transmission between locations Eligibility 
        eligibility_transmission = pd.read_excel(path + "\Eligibility_Transmission.xlsx", 
                                         index_col=0, header=0)
        distances_transmission = pd.read_excel(path + "\Matrix Distance_Transmission.xlsx", 
                                       index_col=0, header=0)
        detour_factor = 1.3
            
            
            ### Check Again ###
        if electricity_transmission == True:
            capex_electricGrid = 0.0014125 # 1412.5 €/MW/km for 2020 [ETRI2014] Powerfactor = 0.8 
            esM.add(fn.Transmission(esM=esM, 
                        name='Electric grid', 
                        commodity='electricity', 
                        hasCapacityVariable=True,
                        #capacityFix=capacityFix,
                        locationalEligibility=eligibility_transmission,
                        distances=distances_transmission*detour_factor, 
                        #losses=0.00001,
                        investPerCapacity= capex_electricGrid,
                        opexPerCapacity=capex_electricGrid*0.015,
                        interestRate=0.08,
                        economicLifetime=60
                           ))
                
        if desalination == True:
            capex_waterPipelines = 0.000053                     # 0.053 €/(m3·a·km)
            opex_waterPipelines = (0.00000023+(0.02*0.01923))   # 0.023 €/(m3·a·100 km) + 2% of 19.23 €/(m3·h·km) pumping
            esM.add(fn.Transmission(esM=esM, 
                            name='Water pipelines', 
                            commodity='freshwater',
                            distances=distances_transmission*detour_factor, 
                            hasCapacityVariable=True, 
                            locationalEligibility=eligibility_transmission,
                            #capacityMax=eligibility_transmission*15,   #New
                            investPerCapacity=capex_waterPipelines,
                            opexPerCapacity=opex_waterPipelines,
                            interestRate=0.08, 
                            economicLifetime=50))
                
        if H2_pipline == True:
            esM.add(fn.Transmission(esM=esM, 
                            name='H2 pipelines', 
                            commodity='hydrogen',
                            distances=distances_transmission*detour_factor, 
                            hasCapacityVariable=True, 
                            locationalEligibility=eligibility_transmission,
                            capacityMax=eligibility_transmission*15,   #New
                            investPerCapacity=0.000185, 
                            opexPerCapacity=1e-6,
                            interestRate=0.08, 
                            economicLifetime=40))
                
        if electricity_demand == True:
            esM.add(fn.Storage(esM=esM, name='Li-ion batteries', 
                       commodity='electricity', 
                       hasCapacityVariable=True,
                       chargeEfficiency=0.95, 
                       cyclicLifetime=10000, 
                       dischargeEfficiency=0.95, 
                       selfDischarge=1-(1-0.03)**(1/(30*24)), 
                       chargeRate=1, 
                       dischargeRate=1, 
                       investPerCapacity=0.151, 
                       opexPerCapacity=0.151*0.01, 
                       interestRate=0.08,
                       economicLifetime=22))
                
        if desalination == True:
                #operationRateFix_freshwater = pd.read_excel(path + "/Freshwater Demand/freshwater_operationRateFix.xlsx", 
    #                                            index_col=0, header=0)
            esM.add(fn.Sink(esM=esM, 
                                name='Water demand', 
                                commodity='freshwater', 
                                hasCapacityVariable=False,
                                #operationRateFix=operationRateFix_freshwater
                               ))
        
        # Export at location 23 & 1_20

        eligibility_export = pd.Series([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 
                                0, 0, 0, 0, 0, 0, 0, 0], index=locations)
        if H2_demand == True:
            operationRateFix_hydrogen = pd.read_excel(path + "\Hydrogen Demand\hydrogen_operationRateFix.xlsx", 
                                              index_col=0, header=0)
            esM.add(fn.Sink(esM=esM, 
                    name='H2 demand', 
                    commodity='hydrogen', 
                    hasCapacityVariable=False,
                    #locationalEligibility=eligibility_export,
                    #commodityRevenue=0.5,
                    operationRateFix=operationRateFix_hydrogen
                   ))
                
        if brine_disposal == True:
            esM.add(fn.Sink(esM=esM, 
                    name='brine disposal', 
                    commodity='brine', 
                    locationalEligibility=eligibility,
                    hasCapacityVariable=False))
            
        esM.cluster(numberOfTypicalPeriods=7, numberOfTimeStepsPerPeriod=8)
            
        esM.optimize(timeSeriesAggregation=True, optimizationSpecs='LogToConsole=1 OptimalityTol=1e-6', solver='gurobi')
            
        df_1 = esM.getOptimizationSummary("SourceSinkModel", outputLevel=0)
        df1=pd.DataFrame(df_1)
        df1.to_excel(r'C:\Users\user\Desktop\Python\Fine-master\My Model\Final Code\Results\Combined\SourceSink_Combined_{}_{}_{}.xlsx'.format(i,j,k))
    
        df_2 = esM.getOptimizationSummary("TransmissionModel", outputLevel=0)
        df2=pd.DataFrame(df_2)
        df2.to_excel(r'C:\Users\user\Desktop\Python\Fine-master\My Model\Final Code\Results\Combined\TransmissionModel_Combined_{}_{}_{}.xlsx'.format(i,j,k))
    
        df_3=esM.getOptimizationSummary("ConversionModel", outputLevel=0)
        df3=pd.DataFrame(df_3)
        df3.to_excel(r'C:\Users\user\Desktop\Python\Fine-master\My Model\Final Code\Results\Combined\ConversionModel_Combined_{}_{}_{}.xlsx'.format(i,j,k))

        df_4=esM.getOptimizationSummary("StorageModel", outputLevel=0)
        df4=pd.DataFrame(df_4)
        df4.to_excel(r'C:\Users\user\Desktop\Python\Fine-master\My Model\Final Code\Results\Combined\StorageModel_Combined_{}_{}_{}.xlsx'.format(i,j,k))

## Results

In [ ]:
esM.getOptimizationSummary("SourceSinkModel", outputLevel=2)

In [ ]:
esM.getOptimizationSummary("StorageModel", outputLevel=2)

In [ ]:
esM.getOptimizationSummary("ConversionModel", outputLevel=2)

In [ ]:
esM.getOptimizationSummary("TransmissionModel", outputLevel=0)

## Output

In [ ]:
if scenario == 'DSWE':
    
    df_1 = esM.getOptimizationSummary("SourceSinkModel", outputLevel=0)
    df1=pd.DataFrame(df_1)
    df1.to_excel(r'C:\Users\user\Desktop\Python\Fine-master\My Model\Final Code\Results\DSWE_1\SourceSink_DSW_1.xlsx')
    
    df_2 = esM.getOptimizationSummary("TransmissionModel", outputLevel=0)
    df2=pd.DataFrame(df_2)
    df2.to_excel(r'C:\Users\user\Desktop\Python\Fine-master\My Model\Final Code\Results\DSWE_1\TransmissionModel_DSW_1.xlsx')
    
    df_3=esM.getOptimizationSummary("ConversionModel", outputLevel=0)
    df3=pd.DataFrame(df_3)
    df3.to_excel(r'C:\Users\user\Desktop\Python\Fine-master\My Model\Final Code\Results\DSWE_1\ConversionModel_DSW_1.xlsx')
    
    df_4=esM.getOptimizationSummary("StorageModel", outputLevel=0)
    df4=pd.DataFrame(df_4)
    df4.to_excel(r'C:\Users\user\Desktop\Python\Fine-master\My Model\Final Code\Results\DSWE_1\StorageModel_DSW_1.xlsx')

elif scenario == 'AWE':
    
    df_1 = esM.getOptimizationSummary("SourceSinkModel", outputLevel=0)
    df1=pd.DataFrame(df_1)
    df1.to_excel(r'C:\Users\user\Desktop\Python\Fine-master\My Model\Final Code\Results\AWE\SourceSink_AWE.xlsx')
    
    df_2 = esM.getOptimizationSummary("TransmissionModel", outputLevel=0)
    df2=pd.DataFrame(df_2)
    df2.to_excel(r'C:\Users\user\Desktop\Python\Fine-master\My Model\Final Code\Results\AWE\TransmissionModel_AWE.xlsx')
    
    df_3=esM.getOptimizationSummary("ConversionModel", outputLevel=0)
    df3=pd.DataFrame(df_3)
    df3.to_excel(r'C:\Users\user\Desktop\Python\Fine-master\My Model\Final Code\Results\AWE\ConversionModel_AWE.xlsx')
    
    df_4=esM.getOptimizationSummary("StorageModel", outputLevel=0)
    df4=pd.DataFrame(df_4)
    df4.to_excel(r'C:\Users\user\Desktop\Python\Fine-master\My Model\Final Code\Results\AWE\StorageModel_AWE.xlsx')

elif scenario == 'Combined':
    
    

In [ ]:
locFilePath = os.path.join(cwd, 'InputData', 'Regions_Egypt','EG_38_Regions.shp')
fig, ax = fn.plotLocations(locFilePath, plotLocNames=True, indexColumn='GID_1', crs='epsg:22993', fontsize=9, 
                           figsize=(9, 9), save=True, fileName='Regions', dpi=500, edgeColor='blue', linewidth=0.75)

In [1]:
locFilePath_PV = os.path.join(cwd, 'InputData', 'Regions_Egypt','EG_38_Regions_location.shp')
fig, ax = fn.plotLocationalColorMap(esM, 'PV', locFilePath_PV, indexColumn='GID_1', crs='epsg:22993', perArea=False)

NameError: name 'os' is not defined

In [ ]:
fig, ax = fn.plotLocationalColorMap(esM, 'AW_Electrolyzer', locFilePath_PV, 'GID_1', perArea=False)

In [ ]:
fig, ax = fn.plotOperationColorMap(esM, 'Li-ion batteries', 'location10', 
                                   variableName='stateOfChargeOperationVariablesOptimum')